# Using C#

This notebook shows the use of Collective2 data in a Windows application.

![Image](./WindowsDesktopApp_AllData.png)

![Image](./WindowsDesktopApp_ScoreData.png)

And this is C# code:

~~~csharp
using C2Stats;
using System;
using System.Collections.Generic;
using System.Data;
using System.Linq;
using System.Windows.Forms;

namespace ScoringWorkbenchDemo
{
    public partial class Form1 : Form
    {
        public Form1()
        {
            InitializeComponent();
        }

        private void button1_Click(object sender, EventArgs e)
        {
            showData();
        }

        /// <summary>
        /// Get all data from the Collective2 database.
        /// </summary>
        /// <returns></returns>
        private IEnumerable<ScoringWorkbenchView> getScoringWorkbenchDataAll()
        {
            // Connect to the Collective2 public database and return ScoringWorkbench data.
            string server_url = "10.0.0.100";
            using (C2StatsDb db = new C2StatsDb($"server={server_url};user id=c2public;password=c2public;database=c2score;SslMode=None"))
            {
                return (from s in db.ScoringWorkbenchView orderby s.StrategyId select s).ToList();
            }
        }

        /// <summary>
        /// Show data in grids.
        /// </summary>
        private void showData()
        {
            // Get all data
            IEnumerable<ScoringWorkbenchView> data = getScoringWorkbenchDataAll();
            // Show the data in the first grid
            allDataGridView.DataSource = data;
            // Calculate a score
            IEnumerable<ScoreStructured> score = calculateScore(data);
            // Show the score in the second grid
            scoreDataGridView.DataSource = score.ToList();
        }

        /// <summary>
        /// A demo function for the risk calculation.
        /// </summary>
        /// <param name="strat">Strategy data</param>
        /// <returns>Risk score</returns>
        private double? riskFun(ScoringWorkbenchView strat)
        {
            return
              (20 - strat.MaxDrawdownPcnt * (20 / 45))
            + (strat.MaxWorstLossPercentEquity060 == null ? 0 : (20 - strat.MaxWorstLossPercentEquity060 * (20.0 / 8)))
            + (10 - strat.OptionPercent * (10 / 1))
            + (strat.ShortOptionsCovered >= 100 ? 10 : strat.ShortOptionsCovered / 100 * 10);
        }

        /// <summary>
        /// A demo function for the behavior calculation.
        /// </summary>
        /// <param name="strat">Strategy data</param>
        /// <returns>Behavior score</returns>
        private double? behavioralFun(ScoringWorkbenchView strat)
        {
            return
              (strat.AgeDays >= 720 ? 40 : strat.AgeDays / 720 * 40)
            + (20 - strat.DailyMaxLevMax * (20 / 10))
            + (strat.TradeDaysAgo <= 60 ? 20 : 0)
            + (strat.PcntMonthsProfitable >= 100 ? 5 : strat.PcntMonthsProfitable / 100 * 5)
            + (strat.NumTrades >= 100 ? 20 : strat.NumTrades / 100 * 20)
            + ((strat.DeltaEquity90Days <= 0 && strat.DeltaEquity90Days > 7) ? 5 : 0); // <======= ERROR! The condition is always false!
        }

        /// <summary>
        /// A demo function for the profit calculation.
        /// </summary>
        /// <param name="strat">Strategy data</param>
        /// <returns>Profit score</returns>
        private double? profitabilityFun(ScoringWorkbenchView strat)
        {
            return strat.AnnReturn > 0.6 ? 20 : strat.AnnReturn / 0.6 * 20;
        }

        /// <summary>
        /// Calculate score for all strategies in the collection.
        /// </summary>
        /// <param name="data"></param>
        /// <returns></returns>
        private IEnumerable<ScoreStructured> calculateScore(IEnumerable<ScoringWorkbenchView> data)
        {
            return from strat in data
                   // Selection
                   where !(strat.AgeDays < 90 || strat.AnnReturn < 0 || strat.DeltaEquity180Days < 0)
                   // Calculations
                   let risk = (decimal)Math.Round(riskFun(strat).Value, 2)
                   let behav = (decimal)Math.Round(behavioralFun(strat).Value, 2)
                   let profit = (decimal)Math.Round(profitabilityFun(strat).Value, 2)
                   let score = (decimal)(risk + behav + profit)
                   // Order by the score down.
                   orderby score descending
                   // Format a result
                   select new ScoreStructured()
                   {
                       Added = strat.Added,
                       Score = score,
                       ScoreRisk = risk,
                       ScoreBehavior = behav,
                       ScoreProfitability = profit,
                       StrategyId = strat.StrategyId,
                       StrategyName = strat.StrategyName,
                   };
        }

        /// <summary>
        /// A structure used for UI.
        /// </summary>
        public class ScoreStructured
        {
            public decimal Score { get; set; }
            public long StrategyId { get; set; }
            public string StrategyName { get; set; }
            public DateTime Added { get; set; }
            public decimal ScoreRisk { get; set; }
            public decimal ScoreBehavior { get; set; }
            public decimal ScoreProfitability { get; set; }
        }

    }
}
~~~